# 03: Standard Library Packages and CI Integration

## Part 1: Standard Library

KindScript ships pre-built architectural patterns as packages. Instead of writing Kind definitions and contracts from scratch, import them:

| Package | Pattern | Layers | Contracts |
|---------|---------|--------|-----------|
| `@kindscript/clean-architecture` | Clean Architecture | domain, application, infrastructure | noDependency, purity |
| `@kindscript/hexagonal` | Hexagonal / Ports & Adapters | domain, ports, adapters | noDependency, mustImplement, purity |
| `@kindscript/onion` | Onion Architecture | core, domainServices, applicationServices, infrastructure | noDependency, purity |

## Part 2: CI Integration

KindScript follows Unix conventions — exit 0 = success, exit 1 = failure. Diagnostics use TypeScript's format. Works with any CI system that can run `node`.

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; stdout: string; stderr: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  return {
    code,
    stdout: new TextDecoder().decode(stdout).trim(),
    stderr: new TextDecoder().decode(stderr).trim(),
  };
}

function kscPrint(...args: string[]): Promise<{ code: number; stdout: string; stderr: string }> {
  return ksc(...args).then(r => {
    const output = (r.stdout + r.stderr).trim();
    if (output) console.log(output);
    console.log(`\nExit code: ${r.code}`);
    return r;
  });
}

const BOILERPLATE = `
interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

type MemberMap<T extends Kind> = {
  [K in keyof T as K extends 'kind' | 'location' ? never : K]:
    T[K] extends Kind
      ? MemberMap<T[K]> | { path: string } & Partial<MemberMap<T[K]>> | Record<string, never>
      : never;
};
function locate<T extends Kind>(root: string, members: MemberMap<T>): MemberMap<T> {
  void root;
  return members;
}

interface ContractConfig {
  noDependency?: [string, string][];
  purity?: string[];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}
`.trimStart();

console.log("Setup complete.");

---

## Package 1: `@kindscript/clean-architecture`

Clean Architecture (Robert C. Martin) separates code into three layers:
- **Domain** — pure business logic, no external dependencies
- **Application** — use case orchestration
- **Infrastructure** — database adapters, HTTP clients, etc.

In [ ]:
// What the package provides
console.log(Deno.readTextFileSync(`${PROJECT_ROOT}/packages/clean-architecture/index.ts`));

The package exports:
- **`CleanContext`** — Kind with `domain`, `application`, `infrastructure` members
- **`cleanArchitectureContracts`** — `noDependency` + `purity` rules
- **`locate`** and **`MemberMap`** — re-exported for convenience

### Using the package

With the stdlib, your `architecture.ts` is minimal — just the `locate<T>()` call:

In [ ]:
const DEMO = Deno.makeTempDirSync({ prefix: "ksc-stdlib-" });
Deno.mkdirSync(`${DEMO}/src/domain`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/application`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/infrastructure`, { recursive: true });

// Install the package (copy from packages/ directory)
Deno.mkdirSync(`${DEMO}/node_modules/@kindscript/clean-architecture`, { recursive: true });
Deno.copyFileSync(
  `${PROJECT_ROOT}/packages/clean-architecture/index.ts`,
  `${DEMO}/node_modules/@kindscript/clean-architecture/index.ts`,
);

Deno.writeTextFileSync(`${DEMO}/src/domain/entity.ts`, `
export interface Order { id: string; total: number; }
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/application/handler.ts`, `
import { Order } from '../domain/entity';
export function process(o: Order): void { console.log(o.id); }
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/infrastructure/repository.ts`, `
import { Order } from '../domain/entity';
export function save(o: Order): void { /* persist */ }
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: ".", outDir: "dist" },
  include: ["src/**/*.ts", "architecture.ts"],
}, null, 2));

// architecture.ts — just a locate<T>() call, types come from the package
Deno.writeTextFileSync(`${DEMO}/architecture.ts`, `
import { CleanContext, locate } from '@kindscript/clean-architecture';

export const app = locate<CleanContext>("src", {
  domain: {},
  application: {},
  infrastructure: {},
});
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/kindscript.json`, JSON.stringify({
  definitions: ["architecture.ts"],
  packages: ["@kindscript/clean-architecture"],
}, null, 2));

console.log("=== architecture.ts (with stdlib) ===");
console.log(Deno.readTextFileSync(`${DEMO}/architecture.ts`));

In [ ]:
// Contracts come from the package — check passes
await kscPrint("check", DEMO);

In [ ]:
// Stdlib contracts still enforce — introduce a violation
Deno.writeTextFileSync(`${DEMO}/src/domain/entity.ts`, `
import { save } from '../infrastructure/repository';

export interface Order { id: string; total: number; }
export function createAndSave(): void { save({ id: '1', total: 0 }); }
`.trimStart());

console.log("=== Violation: domain imports from infrastructure ===");
await kscPrint("check", DEMO);

Deno.removeSync(DEMO, { recursive: true });

### Inline vs. Stdlib comparison

| | Inline | Stdlib |
|---|--------|--------|
| `architecture.ts` size | ~40 lines (boilerplate + definitions + contracts) | ~7 lines (just the locate call) |
| Contracts | You write them | Pre-configured by the package |
| Customization | Full control | Import and extend |

**Recommendation:** Start with the stdlib package, customize later if needed.

---

## Package 2: `@kindscript/hexagonal`

Hexagonal Architecture (Alistair Cockburn) — Ports & Adapters:
- **Domain** — core business logic
- **Ports** — interfaces defining how the domain talks to the outside world
- **Adapters** — implementations of ports for specific technologies

Adds `mustImplement` — every port interface must have an adapter class.

In [ ]:
console.log(Deno.readTextFileSync(`${PROJECT_ROOT}/packages/hexagonal/index.ts`));

### Usage

```typescript
import { HexagonalContext, locate } from '@kindscript/hexagonal';

export const app = locate<HexagonalContext>("src", {
  domain: {},
  ports: {},
  adapters: {},
});
```

---

## Package 3: `@kindscript/onion`

Onion Architecture (Jeffrey Palermo) — concentric rings:
- **Core** — domain model, entities, value objects (innermost)
- **Domain Services** — logic spanning multiple entities
- **Application Services** — use case orchestration
- **Infrastructure** — external concerns (outermost)

Has the most `noDependency` contracts — 5 rules enforcing inward-only dependencies.

In [ ]:
console.log(Deno.readTextFileSync(`${PROJECT_ROOT}/packages/onion/index.ts`));

### Usage

```typescript
import { OnionContext, locate } from '@kindscript/onion';

export const app = locate<OnionContext>("src", {
  core: {},
  domainServices: {},
  applicationServices: {},
  infrastructure: {},
});
```

---

## `ksc infer` with Stdlib

When `ksc infer` detects that a `@kindscript/*` package is installed, it generates import-based definitions instead of inline stubs.

In [ ]:
const INFER_DEMO = Deno.makeTempDirSync({ prefix: "ksc-infer-stdlib-" });
Deno.mkdirSync(`${INFER_DEMO}/src/domain`, { recursive: true });
Deno.mkdirSync(`${INFER_DEMO}/src/application`, { recursive: true });
Deno.mkdirSync(`${INFER_DEMO}/src/infrastructure`, { recursive: true });
Deno.mkdirSync(`${INFER_DEMO}/node_modules/@kindscript/clean-architecture`, { recursive: true });

Deno.copyFileSync(
  `${PROJECT_ROOT}/packages/clean-architecture/index.ts`,
  `${INFER_DEMO}/node_modules/@kindscript/clean-architecture/index.ts`,
);

Deno.writeTextFileSync(`${INFER_DEMO}/src/domain/entity.ts`, `export interface Order { id: string; }\n`);
Deno.writeTextFileSync(`${INFER_DEMO}/src/application/handler.ts`, `import { Order } from '../domain/entity';\nexport function handle(o: Order) {}\n`);
Deno.writeTextFileSync(`${INFER_DEMO}/src/infrastructure/repository.ts`, `import { Order } from '../domain/entity';\nexport function save(o: Order) {}\n`);
Deno.writeTextFileSync(`${INFER_DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: ".", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2));

console.log("=== ksc infer with @kindscript/clean-architecture installed ===");
await kscPrint("infer", INFER_DEMO);

Deno.removeSync(INFER_DEMO, { recursive: true });

Instead of ~40 lines of boilerplate, the generated `architecture.ts` starts with an import and a single `locate<T>()` call.

---

## Part 2: CI Integration

### Exit Code Behavior

| Scenario | Exit code |
|----------|---------|
| All contracts satisfied | `0` |
| One or more violations | `1` |
| Missing config file | `1` |

Same convention as `tsc`, `eslint`, and `jest`.

In [ ]:
function makeCleanProject(): string {
  const dir = Deno.makeTempDirSync({ prefix: "ksc-ci-" });
  Deno.mkdirSync(`${dir}/src/domain`, { recursive: true });
  Deno.mkdirSync(`${dir}/src/infrastructure`, { recursive: true });
  Deno.writeTextFileSync(`${dir}/src/domain/entity.ts`, `export interface User { id: string; }\n`);
  Deno.writeTextFileSync(`${dir}/src/infrastructure/repo.ts`, `import { User } from '../domain/entity';\nexport function save(u: User) {}\n`);
  Deno.writeTextFileSync(`${dir}/tsconfig.json`, JSON.stringify({
    compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
    include: ["src/**/*.ts"],
  }, null, 2));
  Deno.writeTextFileSync(`${dir}/kindscript.json`, JSON.stringify({ definitions: ["architecture.ts"] }, null, 2));
  Deno.writeTextFileSync(`${dir}/architecture.ts`, BOILERPLATE + `
export interface App extends Kind<"App"> {
  domain: DomainLayer;
  infrastructure: InfrastructureLayer;
}
export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app = locate<App>("src", {
  domain: {},
  infrastructure: {},
});

export const contracts = defineContracts<App>({
  noDependency: [["domain", "infrastructure"]],
});
`);
  return dir;
}

// Exit code 0: clean project
const clean = makeCleanProject();
const cleanResult = await ksc("check", clean);
console.log("Clean project exit code:", cleanResult.code);

// Exit code 1: violation
const violation = makeCleanProject();
Deno.writeTextFileSync(`${violation}/src/domain/entity.ts`, `
import { save } from '../infrastructure/repo';
export interface User { id: string; }
`.trimStart());
const violationResult = await ksc("check", violation);
console.log("\nViolation project exit code:", violationResult.code);
console.log("Output:", violationResult.stdout || violationResult.stderr);

Deno.removeSync(clean, { recursive: true });
Deno.removeSync(violation, { recursive: true });

### Diagnostic Output Format

Diagnostics follow the TypeScript format:

```
file:line:column - error KSXXXXX: message
  Contract 'name' (type) defined at location
```

Existing tools that parse TypeScript errors (VS Code problem matchers, CI log parsers) work out of the box.

In [ ]:
const demo = makeCleanProject();
Deno.writeTextFileSync(`${demo}/src/domain/entity.ts`, `
import { save } from '../infrastructure/repo';
export interface User { id: string; }
`.trimStart());

const result = await ksc("check", demo);
const output = result.stdout + result.stderr;

console.log("=== Raw diagnostic output ===");
console.log(output);

console.log("\n=== Parsed ===");
for (const line of output.split("\n")) {
  const match = line.match(/(.+):(\d+):(\d+) - error (KS\d+): (.+)/);
  if (match) {
    console.log("  File:   ", match[1]);
    console.log("  Line:   ", match[2]);
    console.log("  Column: ", match[3]);
    console.log("  Code:   ", match[4]);
    console.log("  Message:", match[5]);
  }
}

Deno.removeSync(demo, { recursive: true });

### GitHub Actions

```yaml
# .github/workflows/ci.yml
name: CI
on: [push, pull_request]

jobs:
  check:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with:
          node-version: 20
      - run: npm ci
      - run: npx tsc --noEmit        # Type check
      - run: npx ksc check            # Architecture check
      - run: npm test                 # Tests
```

### Pre-commit Hook

```bash
# .git/hooks/pre-commit (chmod +x)
#!/bin/sh
npx ksc check || exit 1
```

Or with [Husky](https://typicode.github.io/husky/):

```bash
# .husky/pre-commit
npx ksc check
```

### Migration Workflow

For projects adopting KindScript for the first time:

```bash
npx ksc init --detect              # Step 1: Detect
npx ksc infer --write              # Step 2: Generate definitions
npx ksc check                      # Step 3: Find existing violations
# Edit architecture.ts if needed   # Step 4: Adjust contracts
# Add to CI                        # Step 5: Enforce
```

**Gradual adoption:** Start with just `noDependency` for your most critical boundary. Add more contracts as you clean up.

In [ ]:
const migrate = Deno.makeTempDirSync({ prefix: "ksc-migrate-" });
Deno.mkdirSync(`${migrate}/src/domain`, { recursive: true });
Deno.mkdirSync(`${migrate}/src/application`, { recursive: true });
Deno.mkdirSync(`${migrate}/src/infrastructure`, { recursive: true });

Deno.writeTextFileSync(`${migrate}/src/domain/entity.ts`, `export interface User { id: string; }\n`);
Deno.writeTextFileSync(`${migrate}/src/application/handler.ts`, `import { User } from '../domain/entity';\nexport function handle(u: User) {}\n`);
Deno.writeTextFileSync(`${migrate}/src/infrastructure/repo.ts`, `import { User } from '../domain/entity';\nexport function save(u: User) {}\n`);
Deno.writeTextFileSync(`${migrate}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2));

console.log("=== Step 1: Detect ===");
await kscPrint("init", "--detect", migrate);

console.log("\n=== Step 2: Infer + write ===");
await kscPrint("infer", "--write", migrate);

console.log("\n=== Step 3: Check ===");
await kscPrint("check", migrate);

console.log("\nMigration complete.");

Deno.removeSync(migrate, { recursive: true });

---

## Summary

### Stdlib Packages

| Package | Best for | Key contract |
|---------|----------|--------------|
| `@kindscript/clean-architecture` | Most backend services | `noDependency` between all 3 layers |
| `@kindscript/hexagonal` | Systems with many integrations | `mustImplement` (ports need adapters) |
| `@kindscript/onion` | Complex domain-heavy applications | 5 `noDependency` rules (concentric rings) |

All three include `purity` on the innermost layer.

### CI Integration

| Integration point | How |
|---|---|
| **CI pipeline** | `npx ksc check` — exit 1 on violations |
| **Pre-commit hook** | `npx ksc check \|\| exit 1` |
| **VS Code** | TypeScript language service plugin |
| **npm scripts** | `"arch-check": "ksc check"` in package.json |